In [ ]:
import pandas as pd
from matplotlib import pyplot
%matplotlib inline
import matplotlib
import pandas.tseries
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import _warnings
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import acf
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.arima_model import ARIMA
from scipy.stats import boxcox
import seaborn as sns
sns.set_style('whitegrid')
from matplotlib import colors
import warnings
warnings.filterwarnings('ignore')
from pandas.plotting import autocorrelation_plot
from pandas import concat
from matplotlib.pylab import rcParams
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings

from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib as mpl
%matplotlib inline
import plotly.express as px
import plotly.io as pio
from pandas import Series
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
%matplotlib inline




df = pd.read_csv('zillow_data.csv')

In [ ]:
# preview the Data frame 
df.head()

In [ ]:
#EDA FOR CA
df_ca = df.loc[df['State']=='CA']


In [ ]:
df_ca.drop(['RegionID','City','State','CountyName'],axis=1,inplace=True)

In [ ]:
df_ca.rename(columns={'RegionName':'Zipcode'},inplace=True)

In [ ]:
df_ca

In [ ]:
#ROI - 3 year 
df_ca['ROI_3yr']=round((df_ca['2018-04']-df_ca['2015-01'])/df_ca['2015-01'],4)


In [ ]:
def Melta_df(df):
    melted=pd.melt(df,id_vars=['Zipcode','Metro','SizeRank','ROI_3yr'],var_name='time')
    melted['time']=pd.to_datetime(melted['time'],infer_datetime_format=True)
    melted=melted.dropna(subset=['value'])
    return melted


In [ ]:
Melta_df=Melta_df(df_ca)

In [ ]:
df_ca['Zipcode']=df_ca['Zipcode'].astype(str)

In [ ]:
Melta_df['time']=pd.to_datetime(Melta_df['time'])
Melta_df.set_index('time',inplace=True)

In [ ]:
Melta_df=Melta_df.dropna()

In [ ]:
#made the range time from 2008 
Melta_df=Melta_df['2008-01-01':'2018-04-01']

In [ ]:
Melta_df.resample('MS').mean().index

In [ ]:
Melta_df.info()

In [ ]:
Melta_df.columns

EDA

In [ ]:
# Average Home price By Metro 

In [ ]:
metro_grp_ca=Melta_df.groupby('Metro')
metro_grp_ca=metro_grp_ca.value.mean()
metro_grp_ca=metro_grp_ca.sort_values(ascending=False).head(5)
metro_grp_ca.plot.barh(color='blue',figsize=(15,5))
plt.title('10-Year Mean Value by Metro')
plt.xlabel('Value in Millions of Dollars ($)')
plt.show()
plt.savefig('10yr-metro-mean-value')


In [ ]:
#Home value by month 
monthly_data_ca=Melta_df.resample('MS').mean()['value']
monthly_data_ca.plot(figsize=(15,5))
plt.title('Typical Home Value by Month')
plt.ylabel('Value in US Dollars ($)')
plt.show()

In [ ]:
#Top 5 Zipcodes with highest mean value 
zipcode_grp_ca = Melta_df.groupby('Zipcode')
mean_value_ca =zipcode_grp_ca.value.mean()
mean_value_ca=mean_value_ca.sort_values(ascending=False).head(5)

mean_value_ca.plot.bar(color='blue',figsize=(10,5))
plt.title('Top 5 Zipcodes \n Highest Mean Value 2008-2018')
plt.ylabel('Value in Millions of Dollars ($)')
plt.show()
plt.savefig('Zipcodes')

In [ ]:
#Top 5 zipcodes with the higest 3 year Retrun on Investment 

ninety_perc_ROI_3yr=Melta_df['ROI_3yr'].quantile(q=0.90)
top_5 = Melta_df.loc[Melta_df['ROI_3yr']>=ninety_perc_ROI_3yr]
df_x=top_5.drop(columns=['Metro','SizeRank'])
fig = px.line(df_x,color='Zipcode', labels = {'_value','year'})
fig.show()



In [ ]:
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd

In [ ]:
app = Dash(__name__)

In [ ]:
app.layout = html.Div(children =[
        dcc.Graph(
        figure =fig)
])


In [ ]:
app.layout = html.Div(children=[
    html.H1(children='Top Zipcodes with ROI'),

    dcc.Graph(
        figure=fig
    )])

In [27]:
if __name__ == '__main__':
    app.run_server(debug=False)

# Step 2: Data Preprocessing

In [ ]:
def get_datetimes(df):
    """
    Takes a dataframe:
    returns only those column names that can be converted into datetime objects 
    as datetime objects.
    NOTE number of returned columns may not match total number of columns in passed dataframe
    """
    
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

# Step 4: Reshape from Wide to Long Format

In [ ]:
#had to drop the region id from the group by becasue its do it by mean then 
# the zip code are not a factor any more. need to figure how to put it back after the model 

def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby(['time']).aggregate({'value':'mean'})

In [ ]:
Model_df=melt_data(df)

In [ ]:
Model_df

In [ ]:

Model_df.reset_index(inplace=True)

In [ ]:
Model_df.columns

In [ ]:
Model_df['time'] = pd.to_datetime(Model_df['time'],format='%d/%m/%y')

In [ ]:
Model_df.set_index('time',inplace=True)

In [ ]:
# verifiying the the datetime dytpe 
Model_df.index

In [ ]:
Model_df

In [ ]:
#Checking for Null - not in index and columns! -- No Missing data 

Model_df.index.isna().sum()

In [ ]:
Model_df.info()

In [ ]:
#Change the time frame from 2008-2018 due to the housing buble crash, kept it to 10 years window 
Model_df=Model_df['2008-01-01':'2018-04-01']

In [ ]:
Model_df

In [ ]:
Model_df.info()

# pratice code 
temp_2000_onwards=Model_df['2000':]
print(temp_1985_onwards)

Exploratory Data Analysis

metro_grp=Model_df.groupby('Metro')
metro_grp=metro_grp.value.mean()
metro_grp=metro_grp.sort_values(ascending=False).head(10)
metro_grp.plot.barh(color='blue',figsize=(15,10))
plt.title('10-Year Mean Value by Metro')
plt.xlabel('Value in Millions of Dollars ($)')
plt.show()
plt.savefig('10yr-metro-mean-value')


Overall mean value fluctuation over time

In [ ]:
Monthly_Data=Model_df.resample('MS').mean()['value']
Monthly_Data.plot(figsize=(12,8))
plt.title('Home Value by Month')
plt.ylabel('Value in US Dollars ($)')
plt.show()

Top 5 Zip Codes with the Higest Mean Value Between 2008-2018

In [ ]:
zipcode_grp=Model_df.groupby('Zipcode')
#mean_value=zipcode_grp.groupby('value').mean()
mean_value =zipcode_grp.value.mean()
mean_value=mean_value.sort_values(ascending=False).head(5)

mean_value.plot.bar(color='blue',figsize=(10,5))
plt.title('Top 5 Zipcodes  \n Highest Mean Value 2008-2018')
plt.ylabel('Value in Millions of Dollars ($)')
plt.show()
plt.savefig('Zipcodes')

In [ ]:
Model_df.info()

In [ ]:
Model_df.isna().sum()

In [ ]:
#Visualizing Time series data by line plot 
Model_df.plot(figsize=(16,6))

In [ ]:
#Visualizing Time series data by denisty = it seem that the data is not Gaussian 
Model_df.plot(kind='kde',figsize=(15,10))

Grouping and Visualizing time series by year - showing that from 1996-2006 price went up! from 2007-2011 price went down 

In [ ]:
year_group = Model_df.groupby(pd.Grouper(freq='A'))


Time Series Box and Whisker plots - there is no outliers 

In [ ]:
year_group.boxplot(figsize=(50,100))
plt.show()

Rolling Mean & Standard Deviation showing that the time seires is not stationary 

In [ ]:
Roll_mean = Model_df.rolling(window=8,center=False).mean()
Roll_std = Model_df.rolling(window=8,center=False).std()
fig=plt.figure(figsize=(12,7))
plt.plot(Model_df,color='blue',label='Original')
plt.plot(Roll_mean,color='red',label='Rolling Mean')
plt.plot(Roll_std,color='black',label='Rolling Std')
plt.legend(loc='best')
plt.title('Rooling mean & Standard Deviation')
plt.show(block=False)

Dickey-fuller test show the time seires is not stationary 

In [ ]:
dftest = adfuller(Model_df)

In [ ]:
print(dftest)
# p-value is 0.33 higher than 0.05

Remove trend and seasonality 

Log Transform

In [ ]:

df_log_scaled = Model_df
df_log_scaled['value']=boxcox(df_log_scaled['value'],lmbda=0.0)
plt.figure(figsize=(10,6))
plt.plot(df_log_scaled,color='cornflowerblue')
plt.xlabel('Date',size=12)
plt.ylabel('value',size=12)
plt.title('Transformation',size=14)
plt.show

Removing Moving Average Transform

In [ ]:
#Removing Trend with Moving Average 
moving_avg=df_log_scaled.rolling(window=12).mean()
df_log_scaled_ma=df_log_scaled-moving_avg
df_log_scaled_ma.dropna(inplace=True)
plt.figure(figsize=(10,6))
plt.plot(df_log_scaled_ma,color='cornflowerblue')
plt.xlabel('Date',size=12)
plt.ylabel('Value',size=12)
plt.title('After mooving average',size=14)
plt.show()

Differnce Transform

In [ ]:
#Diffrenence 1 
data_diff_1=df_log_scaled.diff(periods=1)
fig=plt.figure(figsize=(11,7))
plt.plot(data_diff_1,color='blue',label='Value Mean')
plt.legend(loc='best')
plt.title('differnced value Mean')
plt.show(block=False)

In [ ]:
data_diff_1.dropna(inplace=True)

In [ ]:
# Diffrenence 2

data_diff_2 = data_diff_1.diff(periods=1)
fig=plt.figure(figsize=(11,7))
plt.plot(data_diff_2,color='blue',label='Value Mean')
plt.legend(loc='best')
plt.title('differnced value Mean')
plt.show(block=False)


In [ ]:

data_diff_2.dropna(inplace=True)

In [ ]:
data_diff_2.info()

Dickey-Fuller Test 

In [ ]:
dftest1=adfuller(data_diff_1)
print(dftest1)

In [ ]:
dftest2=adfuller(data_diff_2)
print(dftest2)

P-value is far less than 0.05 and Test Statistic is below 0.01 and also all crtical values so 
 the time series is stationary, the Data_diff_2 is much better 

Time Series Decomposition 

In [ ]:

from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = sm.tsa.seasonal_decompose(data_diff_2, model='additive',freq=12)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
# Plot gathered statistics
plt.figure(figsize=(12,8))
plt.subplot(411)
plt.plot(data_diff_2, label='Original', color='blue')
plt.legend(loc='best')
plt.subplot(412)
plt.plot(trend, label='Trend', color='blue')
plt.legend(loc='best')
plt.subplot(413)
plt.plot(seasonal,label='Seasonality', color='blue')
plt.legend(loc='best')
plt.subplot(414)
plt.plot(residual, label='Residuals', color='blue')
plt.legend(loc='best')
plt.tight_layout()



Now The time series is A WHITE NOISE MODEL 

Auto Correlation

In [ ]:
plt.figure(figsize=(12,5))
pd.plotting.autocorrelation_plot(data_diff_2)

Partial Autocorrelation 

In [ ]:
rcParams['figure.figsize']=14,5
plot_pacf(data_diff_2,lags=50);

ARMA MODEL - you can delete this part 

In [ ]:
from statsmodels.tsa.arima_model import ARMA
import statsmodels.api as sm
# Fit an ARMA(1,0) model
mod_arma = ARMA(data_diff_2, order=(1,0))
res_arma = mod_arma.fit()



In [ ]:
# Print out summary information on the fit
print(res_arma.summary())

In [ ]:
print(res_arma.params)

SARIMA MODEL 

In [ ]:
import itertools
import statsmodels.api as sm

p=d=q=range(0,3)
pdq=list(itertools.product(p,d,q))
pdqs = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
ans = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = sm.tsa.statespace.SARIMAX(Model_df,
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            output = mod.fit()
            ans.append([comb, combs, output.aic])
            print('ARIMA {} x {}12 : AIC Calculated ={}'.format(comb, combs, output.aic))
        except:
            continue  


In [ ]:
import itertools
import statsmodels.api as sm

In [ ]:
ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
ans_df.loc[ans_df['aic'].idxmin()]

Grid Search Resluts 
pdq (2,0,1)
pdqs (0,0,0,12)
aic -2758.8

In [ ]:
#Builidng the model
Sarimax=sm.tsa.statespace.SARIMAX(Model_df,order=(2,0,1),seasonal_order=(1,1,1,12))
#Fit the Model
output= Sarimax.fit()
#print output summary 
print(output.summary())
output.plot_diagnostics(figsize=(16,10));

In [ ]:
# Get predictions starting from 2008-01-01 and calculate confidence intervals
pred = output.get_prediction(start=pd.to_datetime('2009-01-01'), dynamic=False)
pred_conf = pred.conf_int()


In [ ]:
pred_conf

In [ ]:
rcParams['figure.figsize'] = 15, 6

# Plot observed values
ax = Model_df['1997':].plot(label='observed')

# Plot predicted values
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=0.9)

# Plot the range for confidence intervals
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='g', alpha=0.5)

# Set axes labels
ax.set_xlabel('Date')
ax.set_ylabel('value')
plt.legend()

plt.show()

In [ ]:
# MSE 
# Get the real and predicted values
Value_forecasted = pred.predicted_mean
Value_truth = Model_df['1999-05-01':]

# Compute the mean square error
mse = ((Value_forecasted - Value_truth) ** 2).mean()
print('The Mean Squared Error of our forecasts is {}'.format(round(mse, 2)))

PRODUCING AND VISUALING FORECATS

In [ ]:
# Get forecast 50 steps ahead in future
prediction = output.get_forecast(steps=50)

# Get confidence intervals of forecasts
pred_conf = prediction.conf_int()

In [ ]:
# Plot future predictions with confidence intervals
ax = Model_df.plot(label='observed', figsize=(10, 10))
prediction.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='k', alpha=0.25)
ax.set_xlabel('Date')
ax.set_ylabel('Values Levels')

plt.legend()
plt.show()

# Step 6: Interpreting Results

This model is purely based on time series so the predictions may not be very accurate because apparently there are many other factors that impact the house price such as economics, interest rate, house market supply, etc.
A linear model would be by far more ideal if we want to take other factors into consideration.